# 03. sedデータの前処理

【このノートで実施すること】</br>
　機械学習に使用するため、テーブルデータの中身を精査し、データ型を変更したり、欠損している値を補完したりします。</br>
　このノートではsedデータについて処理します。</br>
</br>
【入力】</br>
　・data/02_format_data/df_sed.plk</br>
</br>
【出力】</br>
　・data/03_preprocessed_data/df_sed.pkl</br>
</br>

## 03.1. 準備

必要なモジュールをインポートしたり、データを分析するメソッドを用意したり、データを読み込んだりしています。

In [ ]:
# 必要なモジュールをインポート
import pandas as pd
import pickle
import os

In [ ]:
# データの分析結果を出すメソッドを用意
def make_analysis_df(df):

    # 分析用の情報を取得
    num_non_empty = df.count()
    num_empty = df.isna().sum()
    empty_percentage = (num_empty / len(df)) * 100
    data_types = df.dtypes
    unique_counts = df.nunique()
    
    # 新しいDataFrameを作成
    analysis_df = pd.DataFrame({
        'カラム名': df.columns,
        '空でないデータ数': num_non_empty.values,
        '空白データ数': num_empty.values,
        '空白率（％）': empty_percentage.values,
        'データ型': data_types.values,
        'ユニークな値の数': unique_counts.values
    })
    return analysis_df

In [ ]:
# 全カラム、全レコードを表示するように設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# データを読み込む
with open('data/02_format_data/df_sed.pkl', 'rb') as file:
    df_sed = pickle.load(file)

## 03.2. データを確認する

データの中身を確認します

In [ ]:
# 先頭の数行を表示
df_sed.head()

In [ ]:
# 分析結果を表示する
df_sed_analyzed_before = make_analysis_df(df_sed)
df_sed_analyzed_before

## 03.3. 前処理

必要に応じてデータを加工します。<br>
各項目の意味についてはJRDBの仕様書を参照ください。<br>
<br>
引用：<br>
　成績データ仕様　第4版α：http://www.jrdb.com/program/Sed/sed_doc.txt<br>
　成績データ仕様書内容の説明：http://www.jrdb.com/program/Sed/sedsiyo_doc.txt<br>
　ＪＲＤＢデータコード表：http://www.jrdb.com/program/jrdb_code.txt<br>

In [ ]:
df_sed['馬番'] = pd.to_numeric(df_sed['馬番'], errors='coerce')
df_sed['馬番'] = df_sed['馬番'].astype(int)

df_sed['競走成績キー'] = df_sed['競走成績キー_血統登録番号'] + df_sed['競走成績キー_年月日']

df_sed['レース条件_距離'] = pd.to_numeric(df_sed['レース条件_距離'], errors='coerce')
df_sed['レース条件_距離'] = df_sed['レース条件_距離'].astype(int)

df_sed['レース条件_トラック情報_芝ダ障害コード'] = df_sed['レース条件_トラック情報_芝ダ障害コード'].astype('category')

df_sed['レース条件_トラック情報_右左'] = df_sed['レース条件_トラック情報_右左'].astype('category')

df_sed['レース条件_トラック情報_内外'] = df_sed['レース条件_トラック情報_内外'].astype('category')

df_sed['レース条件_馬場状態'] = df_sed['レース条件_馬場状態'].astype('category')

df_sed['レース条件_種別'] = df_sed['レース条件_種別'].astype('category')

df_sed['レース条件_条件'] = df_sed['レース条件_条件'].astype('category')

# 'レース条件_記号' 列を3つの新しいカラムに分割
df_sed['レース条件_記号1'] = df_sed['レース条件_記号'].str[0]
df_sed['レース条件_記号2'] = df_sed['レース条件_記号'].str[1:3]
df_sed['レース条件_記号3'] = df_sed['レース条件_記号'].str[3:]
# 'レース条件_記号' 列を削除
df_sed.drop(columns=['レース条件_記号'], inplace=True)
# カテゴリ型に変換
df_sed['レース条件_記号1'] = df_sed['レース条件_記号1'].astype('category')
df_sed['レース条件_記号2'] = df_sed['レース条件_記号2'].astype('category')
df_sed['レース条件_記号3'] = df_sed['レース条件_記号3'].astype('category')

df_sed['レース条件_重量'] = df_sed['レース条件_重量'].astype('category')

df_sed['レース条件_頭数'] = pd.to_numeric(df_sed['レース条件_頭数'], errors='coerce')
mean_value = df_sed['レース条件_頭数'].mean()
df_sed['レース条件_頭数'] = df_sed['レース条件_頭数'].fillna(mean_value).astype(float)

df_sed['馬成績_着順'] = pd.to_numeric(df_sed['馬成績_着順'], errors='coerce')
mean_value = df_sed['馬成績_着順'].mean()
df_sed['馬成績_着順'] = df_sed['馬成績_着順'].fillna(mean_value).astype(int)

df_sed['馬成績_異常区分'] = df_sed['馬成績_異常区分'].astype('category')

# ------------------
# '馬成績_タイム' カラムを秒に変換する関数
def convert_time_to_seconds(time_str):
    # 空白文字が含まれている場合の処理
    if time_str.strip() == '' or ' ' in time_str:
        return float(9 * 60 + 999 / 10.0)
    # 分と秒の部分を抽出
    minutes = int(time_str[0])
    seconds = int(time_str[1:])    
    # 秒に変換して返す（0.1秒単位から秒単位に変換）
    return float(minutes * 60 + seconds / 10.0)

# '馬成績_タイム' カラムを変換
df_sed['馬成績_タイム'] = df_sed['馬成績_タイム'].apply(convert_time_to_seconds)
# ------------------

df_sed['馬成績_斤量'] = pd.to_numeric(df_sed['馬成績_斤量'], errors='coerce')
df_sed['馬成績_斤量'] = df_sed['馬成績_斤量'].astype(float)

df_sed['馬成績_確定単勝オッズ'] = pd.to_numeric(df_sed['馬成績_確定単勝オッズ'], errors='coerce')
df_sed['馬成績_確定単勝オッズ'] = df_sed['馬成績_確定単勝オッズ'].astype(float)

df_sed['馬成績_確定単勝人気順位'] = pd.to_numeric(df_sed['馬成績_確定単勝人気順位'], errors='coerce')
mean_value = df_sed['馬成績_確定単勝人気順位'].mean()
df_sed['馬成績_確定単勝人気順位'] = df_sed['馬成績_確定単勝人気順位'].fillna(mean_value).astype(int)

df_sed['ＪＲＤＢデータ_ＩＤＭ'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_ＩＤＭ'], errors='coerce')
df_sed['ＪＲＤＢデータ_ＩＤＭ'] = df_sed['ＪＲＤＢデータ_ＩＤＭ'].astype(float)

df_sed['ＪＲＤＢデータ_素点'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_素点'], errors='coerce')
df_sed['ＪＲＤＢデータ_素点'] = df_sed['ＪＲＤＢデータ_素点'].astype(float)
df_sed['ＪＲＤＢデータ_馬場差'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_馬場差'], errors='coerce')
df_sed['ＪＲＤＢデータ_馬場差'] = df_sed['ＪＲＤＢデータ_馬場差'].astype(float)
df_sed['ＪＲＤＢデータ_ペース'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_ペース'], errors='coerce')
df_sed['ＪＲＤＢデータ_ペース'] = df_sed['ＪＲＤＢデータ_ペース'].astype(float)

df_sed['ＪＲＤＢデータ_コース取り'] = df_sed['ＪＲＤＢデータ_コース取り'].astype('category')

df_sed['ＪＲＤＢデータ_上昇度コード'] = df_sed['ＪＲＤＢデータ_上昇度コード'].astype('category')

df_sed['ＪＲＤＢデータ_クラスコード'] = df_sed['ＪＲＤＢデータ_クラスコード'].astype('category')

df_sed['ＪＲＤＢデータ_馬体コード'] = df_sed['ＪＲＤＢデータ_馬体コード'].astype('category')

df_sed['ＪＲＤＢデータ_気配コード'] = df_sed['ＪＲＤＢデータ_気配コード'].astype('category')

df_sed['ＪＲＤＢデータ_レースペース'] = df_sed['ＪＲＤＢデータ_レースペース'].astype('category')

df_sed['ＪＲＤＢデータ_馬ペース'] = df_sed['ＪＲＤＢデータ_馬ペース'].astype('category')

df_sed['ＪＲＤＢデータ_テン指数'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_テン指数'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_テン指数'].mean()
df_sed['ＪＲＤＢデータ_テン指数'] = df_sed['ＪＲＤＢデータ_テン指数'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_上がり指数'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_上がり指数'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_上がり指数'].mean()
df_sed['ＪＲＤＢデータ_上がり指数'] = df_sed['ＪＲＤＢデータ_上がり指数'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_ペース指数'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_ペース指数'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_ペース指数'].mean()
df_sed['ＪＲＤＢデータ_ペース指数'] = df_sed['ＪＲＤＢデータ_ペース指数'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_レースＰ指数'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_レースＰ指数'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_レースＰ指数'].mean()
df_sed['ＪＲＤＢデータ_レースＰ指数'] = df_sed['ＪＲＤＢデータ_レースＰ指数'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_1(2)着タイム差'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_1(2)着タイム差'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_1(2)着タイム差'].mean()
df_sed['ＪＲＤＢデータ_1(2)着タイム差'] = df_sed['ＪＲＤＢデータ_1(2)着タイム差'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_前３Ｆタイム'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_前３Ｆタイム'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_前３Ｆタイム'].mean()
df_sed['ＪＲＤＢデータ_前３Ｆタイム'] = df_sed['ＪＲＤＢデータ_前３Ｆタイム'].fillna(mean_value).astype(float)

df_sed['ＪＲＤＢデータ_後３Ｆタイム'] = pd.to_numeric(df_sed['ＪＲＤＢデータ_後３Ｆタイム'], errors='coerce')
mean_value = df_sed['ＪＲＤＢデータ_後３Ｆタイム'].mean()
df_sed['ＪＲＤＢデータ_後３Ｆタイム'] = df_sed['ＪＲＤＢデータ_後３Ｆタイム'].fillna(mean_value).astype(float)

df_sed['確定複勝オッズ下'] = pd.to_numeric(df_sed['確定複勝オッズ下'], errors='coerce')
mean_value = df_sed['確定複勝オッズ下'].mean()
df_sed['確定複勝オッズ下'] = df_sed['確定複勝オッズ下'].fillna(mean_value).astype(float)

df_sed['10時単勝オッズ'] = pd.to_numeric(df_sed['10時単勝オッズ'], errors='coerce')
mean_value = df_sed['10時単勝オッズ'].mean()
df_sed['10時単勝オッズ'] = df_sed['10時単勝オッズ'].fillna(mean_value).astype(float)

df_sed['10時複勝オッズ'] = pd.to_numeric(df_sed['10時複勝オッズ'], errors='coerce')
mean_value = df_sed['10時複勝オッズ'].mean()
df_sed['10時複勝オッズ'] = df_sed['10時複勝オッズ'].fillna(mean_value).astype(float)

df_sed['馬体重'] = pd.to_numeric(df_sed['馬体重'], errors='coerce')
mean_value = df_sed['馬体重'].mean()
df_sed['馬体重'] = df_sed['馬体重'].fillna(mean_value).astype(float)

df_sed['馬体重増減'] = pd.to_numeric(df_sed['馬体重増減'], errors='coerce')
mean_value = df_sed['馬体重増減'].mean()
df_sed['馬体重増減'] = df_sed['馬体重増減'].fillna(mean_value).astype(float)

df_sed['天候コード'] = df_sed['天候コード'].astype('category')

df_sed['コース'] = df_sed['コース'].astype('category')

df_sed['レース脚質'] = df_sed['レース脚質'].astype('category')

df_sed['払戻データ_単勝'] = pd.to_numeric(df_sed['払戻データ_単勝'], errors='coerce')
df_sed['払戻データ_単勝'] = df_sed['払戻データ_単勝'].fillna(0).astype(int)

df_sed['払戻データ_複勝'] = pd.to_numeric(df_sed['払戻データ_複勝'], errors='coerce')
df_sed['払戻データ_複勝'] = df_sed['払戻データ_複勝'].fillna(0).astype(int)

df_sed['本賞金'] = pd.to_numeric(df_sed['本賞金'], errors='coerce')
df_sed['本賞金'] = df_sed['本賞金'].fillna(0).astype(int)

df_sed['収得賞金'] = pd.to_numeric(df_sed['収得賞金'], errors='coerce')
df_sed['収得賞金'] = df_sed['収得賞金'].fillna(0).astype(int)

df_sed['レースペース流れ'] = df_sed['レースペース流れ'].astype('category')
df_sed['馬ペース流れ'] = df_sed['馬ペース流れ'].astype('category')

df_sed['４角コース取り'] = df_sed['４角コース取り'].astype('category')

## 03.4. 加工後のデータを確認する

データが意図した通りに加工されているか確認します

In [ ]:
# 先頭の数行を表示
df_sed.head()

In [ ]:
# 分析結果を表示する
df_sed_analyzed_after = make_analysis_df(df_sed)
df_sed_analyzed_after

## 03.5. データを保存する

加工したデータを保存します

In [ ]:
# ファイルをpickle形式で保存する
def save_dataframe_to_pickle(dataframe, folder_path, file_name):
    
    print("前処理したデータをファイルに保存します")

    # フォルダが存在しない場合は作成
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # ファイルのフルパスを構築
    file_path = os.path.join(folder_path, file_name + ".pkl")

    # DataFrame を pickle 形式で保存
    with open(file_path, 'wb') as file:
        pickle.dump(dataframe, file)

    print("保存しました")

In [ ]:
dataframe = df_sed
folder_path = "data/03_preprocessed_data"
file_name = "df_sed"
save_dataframe_to_pickle(dataframe, folder_path, file_name)